# NNsight 0.3 - User Guide

## Set up

In [1]:
from IPython.display import clear_output

!pip install nnsight
!pip install --upgrade transformers torch

clear_output()

In [2]:
from google.colab import userdata
from nnsight import CONFIG

from nnsight.logger import remote_logger
remote_logger.propagate = False

CONFIG.set_default_api_key('422220a9817141e49c5add1868af07a5')

In [3]:
from collections import OrderedDict
from nnsight import NNsight
import torch

input_size = 5
hidden_dims = 10
output_size = 2

torch.manual_seed(423)

net = torch.nn.Sequential(
    OrderedDict(
        [
            ("layer1", torch.nn.Linear(input_size, hidden_dims)),
            ("layer2", torch.nn.Linear(hidden_dims, hidden_dims))
        ]
    )
).requires_grad_(False)

input = torch.rand((1, input_size))
input_2 = torch.rand((1, input_size))

tiny_model = NNsight(net)

In [4]:
from nnsight import LanguageModel

lm = LanguageModel("openai-community/gpt2", dispatch=True)
llm = LanguageModel("meta-llama/Meta-Llama-3.1-8B")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# Breaking Changes

## input/inputs

Module input access has a syntactic change:

- Old: `nnsight.Envoy.input`

- New: `nnsight.Envoy.inputs`

- Note: `nnsight.Envoy.input` now provides access to the first positional argument of the module's input.

In [6]:
with lm.trace("Hello World"):
  l2_ins = lm.transformer.h[2].inputs.save()
  l2_in = lm.transformer.h[2].input.save()

print("Inputs: ", l2_ins)
print("First Positional Argument Input: ", l2_in)

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Inputs:  ((tensor([[[ 1.0967, -1.8792,  1.0121,  ..., -1.0442, -0.5490, -1.1239],
         [-0.2300,  0.5204,  0.4756,  ..., -1.6795, -0.5285,  0.4190]]],
       grad_fn=<AddBackward0>),), {'layer_past': None, 'attention_mask': None, 'head_mask': None, 'encoder_hidden_states': None, 'encoder_attention_mask': None, 'use_cache': True, 'output_attentions': False})
First Positional Argument Input:  tensor([[[ 1.0967, -1.8792,  1.0121,  ..., -1.0442, -0.5490, -1.1239],
         [-0.2300,  0.5204,  0.4756,  ..., -1.6795, -0.5285,  0.4190]]],
       grad_fn=<AddBackward0>)


## `scan` and `validate`

`scan` and `validate` are now set to `False` by default in the `Tracer` context.

# New Features

### Scanning

You can scan a model without executing it to gather important insights. This is useful for looking at internal modules' shapes for example. You can pass a dummy input to the model, and it will not be executed. This is also means that you don't have to call `save()` on any variable.

In [ ]:
with tiny_model.scan(torch.tensor([0, 0, 0, 0, 0])):
  dim = tiny_model.layer2.input.shape

print(dim)

torch.Size([10])


## nnsight builtins

You can now define multiple `Python` builtins to be traceable by the Intervention graph.

Simply use the `nnsight` import to call constructors for these data structures.

In [8]:
import nnsight

with tiny_model.trace(input):
  num = nnsight.int(5).save()
  l = nnsight.list().save()
  l.append(num)
  d = nnsight.dict({"five": num}).save()

print("Interger: ", num)
print("List: ", l)
print("Dictionary: ", d)

Interger:  5
List:  [5]
Dictionary:  {'five': 5}


Here is the complete list of supported `Python` builtins:
- bool
- bytes
- int
- float
- str
- complex
- bytearray
- tuple
- list
- set
- dict

## Proxy Update

For literals created and traced by `nnsight`, there is no direct way of setting their values.

Use our `.update()` method on Intervention Proxies to assign it a new value.

In [ ]:
import nnsight

with tiny_model.trace(input):
  input_str = nnsight.str("I am a ").save()
  input_str.update(input_str + "Transformer")

print("Input: ", input_str)

Input:  I am a Transformer


This is also useful for calculating running sums and other statistics.

## Logging

We are probably all guilty, at least once, of trying to print an Intervention Proxy from within the tracing context to look at its value:

In [ ]:
with tiny_model.trace(input):
    print(tiny_model.layer1.output)

InterventionProxy (InterventionProtocol_0): 


The reason this does not print any actual value is because the model is only executed upon exiting the `Tracer` context, and thus, the proxies' values have not been populated yet.

If you are still only interested in looking at some intermediate values without necessary saving them, you can call our logging feature which will be executed as an `nnsight` node during the model's execution and show you the actual values.

In [ ]:
import nnsight

with tiny_model.trace(input) as tracer:
  nnsight.log("Layer 1 - out: ", tiny_model.layer1.output)

Layer 1 - out:  tensor([[ 7.2426e-01,  2.4406e-01, -5.3356e-01,  2.4396e-01, -4.4996e-04,
         -1.3551e-01,  1.8728e-01,  7.5127e-01, -3.3102e-01, -8.0205e-01]])


## Tracing function calls

Everything within the tracing context operates on the intervention graph. Therefore for `nnsight` to trace a function it must also be a part of the intervention graph.

Out-of-the-box `nnsight` supports `Pytorch` functions and methods, all operators, as well the `einops` library. We don’t need to do anything special to use them.

For custom functions we can use `nnsight.apply()` to add them to the intervention graph.

In [14]:
import nnsight
import torch

# We define a simple custom function that sums all the elements of a tensor
def tensor_sum(tensor):
    flat = tensor.flatten()
    total = 0
    for element in flat:
        total += element.item()

    return torch.tensor(total)

with lm.trace("The Eiffel Tower is in the city of") as tracer:

    # Specify the function name and its arguments (in a coma-separated form) to add to the intervention graph
    custom_sum = nnsight.apply(tensor_sum, lm.transformer.h[0].output[0]).save()
    sum = lm.transformer.h[0].output[0].sum().save()

print("PyTorch sum: ", sum)
print("Our sum: ", custom_sum)

PyTorch sum:  tensor(191.2440, grad_fn=<SumBackward0>)
Our sum:  tensor(191.2442)


## Early Stopping

If you are only interested in a model's intermediate computations, you can halt a forward pass run at any module level, reducing runtime and conserving computational resources. This is particularly useful if you are working with SAEs.

In [ ]:
with tiny_model.trace(input):
   l1_out = tiny_model.layer1.output.save()
   tiny_model.layer1.output.stop()

print("L1 - Output: ", l1_out)

L1 - Output:  tensor([[ 7.2426e-01,  2.4406e-01, -5.3356e-01,  2.4396e-01, -4.4996e-04,
         -1.3551e-01,  1.8728e-01,  7.5127e-01, -3.3102e-01, -8.0205e-01]])


Interventions within the `Tracer` context do not necessarily execute in the order they are defined. Instead, their execution is tied to the module they are associated with.

As a result, if the forward pass is terminated early any interventions linked to modules beyond that point will be skipped, even if they were defined earlier in the context.

In the example below, the output of layer 2 **CANNOT** be accessed since the model's execution was stopped at layer 1.

In [ ]:
with tiny_model.trace(input):
   l2_out = tiny_model.layer2.output.save()
   tiny_model.layer1.output.stop()

print("L2 - Output: ", l2_out)

L2 - Output:  

ValueError: Accessing value before it's been set.

## Conditional Interventions

You can make interventions conditional!

Create a Conditional context and pass it a value to be evaluated as a boolean. The context will wrap all the interventions that you wish to be dependent on the condition specified.

Let's take a look at how you can do that:

In [ ]:
with tiny_model.trace(input) as tracer:

  rand_int = torch.randint(low=-10, high=10, size=(1,)).item()

  with tracer.cond(rand_int % 2 == 0):
    tracer.apply(print, "Random Integer ", rand_int, " is Even")

  with tracer.cond(rand_int % 2 == 1):
    tracer.apply(print, "Random Integer ", rand_int, " is Odd")

Random Integer  -1  is Odd


In the example above, we have two Conditional contexts with mutually exclusive conditions, mimicking a conventional `If`-`Else` statement.

The condition passed to the Conditional context is evaluated directly by calling `bool()` on the proxy value, so be mindful of how your Intervention Proxy condition evaluates to boolean.

In [ ]:
with tiny_model.trace(input) as tracer:
  l1_out = tiny_model.layer1.output
  with tracer.cond(l1_out != 1):
    tracer.apply(print, "Condition is True")

RuntimeError: Above exception when execution Node: 'ne_0' in Graph: '136573620465632'

The code above throws the **ERROR**: `Boolean value of Tensor with more than one value is ambiguous`, because the condition specified in the Conditional context cannot be handled properly.

Instead, use something like this:

In [ ]:
with tiny_model.trace(input) as tracer:
  l1_out = tiny_model.layer1.output
  with tracer.cond(torch.all(l1_out != 1)):
    tracer.apply(print, "Condition is True")

Condition is True


Conditional contexts can also be nested, if you want your interventions to depend on more than one condition at a time.

In [ ]:
with tiny_model.trace(input) as tracer:
  rand_int = tracer.apply(int, 6)
  with tracer.cond(rand_int > 0):
    with tracer.cond(rand_int % 2 == 0):
      tracer.apply(print, "Rand Int ", rand_int, " is Positive and Even")

Rand Int  6  is Positive and Even


## Model Editing

You can alter a model by setting default edits and interventions in an editing context, applied before each forward pass. This can be used to attach additional modules like SAEs.



In [ ]:
with tiny_model.edit() as edited_model:
  tiny_model.layer1.output[0][:] = 0

with tiny_model.trace(input):
  l1_out = tiny_model.layer1.output.save()

with edited_model.trace(input):
  l1_out_edited = edited_model.layer1.output.save()

print("L1 - Out: ", l1_out)
print("L1 - Out [edited]: ", l1_out_edited)

L1 - Out:  tensor([[ 7.2426e-01,  2.4406e-01, -5.3356e-01,  2.4396e-01, -4.4996e-04,
         -1.3551e-01,  1.8728e-01,  7.5127e-01, -3.3102e-01, -8.0205e-01]])
L1 - Out [edited]:  tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])


Let's look at anotehr example

In [ ]:
from nnsight.util import WrapperModule

class ComplexModule(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.one = WrapperModule()

    def forward(self, x):
        return self.one(x)

l0 = lm.transformer.h[0]
l0.attachment = ComplexModule()

with lm.edit() as gpt2_edited:
    acts = l0.output[0]
    l0.output[0][:] = l0.attachment(acts, hook=True)

# Get values pre editing
with lm.trace("Madison Square Garden is located in the city of"):
    original = l0.output[0].clone().save()
    l0.output[0][:] *= 0.
    original_output = lm.output.logits.save()

with gpt2_edited.trace("Madison Square Garden is located in the city of"):
    one = l0.attachment.one.output.clone().save()
    l0.attachment.output *= 0.
    edited_output = lm.output.logits.save()

print("Original output: ", original_output)
print("Edited output: ", edited_output)

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Original output:  tensor([[[-23.8122, -24.2187, -27.4893,  ..., -30.6782, -29.9407, -24.9513],
         [-23.8122, -24.2187, -27.4893,  ..., -30.6782, -29.9407, -24.9513],
         [-23.8122, -24.2187, -27.4893,  ..., -30.6782, -29.9407, -24.9513],
         ...,
         [-23.8122, -24.2187, -27.4893,  ..., -30.6781, -29.9407, -24.9512],
         [-23.8122, -24.2187, -27.4893,  ..., -30.6781, -29.9407, -24.9512],
         [-23.8122, -24.2187, -27.4893,  ..., -30.6781, -29.9407, -24.9512]]],
       grad_fn=<UnsafeViewBackward0>)
Edited output:  tensor([[[-23.8122, -24.2187, -27.4893,  ..., -30.6782, -29.9407, -24.9513],
         [-23.8122, -24.2187, -27.4893,  ..., -30.6782, -29.9407, -24.9513],
         [-23.8122, -24.2187, -27.4893,  ..., -30.6782, -29.9407, -24.9513],
         ...,
         [-23.8122, -24.2187, -27.4893,  ..., -30.6781, -29.9407, -24.9512],
         [-23.8122, -24.2187, -27.4893,  ..., -30.6781, -29.9407, -24.9512],
         [-23.8122, -24.2187, -27.4893,  ..., -30.6

Your edit call can be customized by choosing to perform edits in-place on the model andgetting access to the editor context (`nnsight.context.Tracer`).

You can also choose to remove edits perfomerd on a model at a later stage.

In [ ]:
with tiny_model.edit(inplace=True, return_context=True) as editor:
  tiny_model.layer1.output[0][:] = 0

with tiny_model.trace(input):
  l1_out = tiny_model.layer1.output.save()

print("L1 - Out: ", l1_out)

tiny_model.clear_edits()

with tiny_model.trace(input):
  l1_out = tiny_model.layer1.output.save()

print("L1 - Out [unedited]: ", l1_out)

L1 - Out:  tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
L1 - Out [unedited]:  tensor([[ 7.2426e-01,  2.4406e-01, -5.3356e-01,  2.4396e-01, -4.4996e-04,
         -1.3551e-01,  1.8728e-01,  7.5127e-01, -3.3102e-01, -8.0205e-01]])


Note that setting new modules with remote execution is currently not supported!

## Session Context

`nnsight 0.3` focuses on enhancing the capabilities of our remote execution API, powered by the [NDIF](https://ndif.us) backend.

To achieve this, we introduce the **Session** context: an overarching structure for efficiently handling multi-tracing experiments. This means that, multiple `Tracer` contexts can be packaged together as part of one single request to the server.

The `Session` context can also be used entirely for local usage, as it enables useful functionalities and optimizes experiments.

In [ ]:
with llm.session(remote=True):
  with llm.trace("_") as t1:
    # define interventions here
    pass

  with llm.trace("_") as t2:
    # define interventions here
    pass

  with llm.trace("_") as t3:
    # define interventions here
    pass

2024-08-30 19:58:12,517 MainProcess nnsight_remote INFO     bf66301e-f693-4146-a721-b4d93c6d3318 - RECEIVED: Your job has been received and is waiting approval.
2024-08-30 19:58:12,544 MainProcess nnsight_remote INFO     bf66301e-f693-4146-a721-b4d93c6d3318 - APPROVED: Your job was approved and is waiting to be run.
2024-08-30 19:58:12,569 MainProcess nnsight_remote INFO     bf66301e-f693-4146-a721-b4d93c6d3318 - RUNNING: Your job has started running.
2024-08-30 19:58:12,684 MainProcess nnsight_remote INFO     bf66301e-f693-4146-a721-b4d93c6d3318 - COMPLETED: Your job has been completed.


All operations defined within a `Session` context are executed at the very end (upon exiting the overarching context) and it is conducted sequentially, strictly following the order of definition (`t2` being executed after `t1` and `t3` after `t2`).

In a `Session`, interventions defined at any early stage can be seamlessly referenced.

In [ ]:
with llm.session(remote=True) as session:
  with llm.trace("The Eiffel Tower is in the city of") as t1:
    hs_11 = llm.model.layers[-1].output[0][:, -1, :] # no .save()
    t1_tokens_out = llm.output.save()

  with llm.trace("Buckingham Palace is in the city of") as t2:
    llm.model.layers[-2].output[0][:, -1, :] = hs_11[:]
    t2_tokens_out = llm.output.save()

print("\nT1 - Prediction: ", llm.tokenizer.decode(t1_tokens_out["logits"].argmax(dim=-1)[0][-1]))
print("T2 - Prediction: ", llm.tokenizer.decode(t2_tokens_out["logits"].argmax(dim=-1)[0][-1]))

2024-08-30 19:56:44,378 MainProcess nnsight_remote INFO     e353b8a7-14a1-4ebe-a477-3ec6ae719726 - RECEIVED: Your job has been received and is waiting approval.
2024-08-30 19:56:44,413 MainProcess nnsight_remote INFO     e353b8a7-14a1-4ebe-a477-3ec6ae719726 - APPROVED: Your job was approved and is waiting to be run.
2024-08-30 19:56:44,496 MainProcess nnsight_remote INFO     e353b8a7-14a1-4ebe-a477-3ec6ae719726 - RUNNING: Your job has started running.
2024-08-30 19:56:45,043 MainProcess nnsight_remote INFO     e353b8a7-14a1-4ebe-a477-3ec6ae719726 - COMPLETED: Your job has been completed.



T1 - Prediction:   Paris
T2 - Prediction:   


In the example above, we are interested in patching the hidden state of a later layer into an earlier one. This experiment can only be conducted with two `Tracer` contexts; since we are using a `Session`, it is not required to save the hidden state from Tracer 1 to reference it in Tracer 2.

The `Session` context can also be terminated early.

In [ ]:
import nnsight

with llm.session(remote=True) as session:
  l = nnsight.list().save()

  l.append(0)
  l.append(1)
  nnsight.log("-- Early Stop --")
  session.exit()
  l.append(2)

print("List: ", l)

2024-08-30 19:58:17,651 MainProcess nnsight_remote INFO     546f8b1b-0f80-47dd-9160-2f6c3d23f154 - RECEIVED: Your job has been received and is waiting approval.
2024-08-30 19:58:17,679 MainProcess nnsight_remote INFO     546f8b1b-0f80-47dd-9160-2f6c3d23f154 - APPROVED: Your job was approved and is waiting to be run.
2024-08-30 19:58:17,694 MainProcess nnsight_remote INFO     546f8b1b-0f80-47dd-9160-2f6c3d23f154 - RUNNING: Your job has started running.
2024-08-30 19:58:17,713 MainProcess nnsight_remote INFO     546f8b1b-0f80-47dd-9160-2f6c3d23f154 - LOG: -- Early Stop --
2024-08-30 19:58:17,750 MainProcess nnsight_remote INFO     546f8b1b-0f80-47dd-9160-2f6c3d23f154 - COMPLETED: Your job has been completed.

List:  [0, 1]


## Iterator Context

We mention earlier that the `Session` context enables multi-tracing execution. But how can we optimize a process that would require running an intervention graph in a loop?

If you create a `for` loop with a `Tracer` context inside of it, this will result in creating a new intervention graph at each iteration, which is not scalable.

We solve this problem the `nnsight` way by introducing the **Iterator** context: an intervention loop that iteratively executes a single intervention graph with an updated parameter.



In [ ]:
import nnsight

with llm.session(remote=True) as session:

  prompts = nnsight.list(["This is nnsight 0.3",
                          "It works with NDIF",
                          "pip install it now!"])
  results = nnsight.list().save()
  with session.iter(prompts) as prompt:

    with llm.trace(prompt):
      results.append(llm.lm_head.output)

2024-08-30 20:17:59,080 MainProcess nnsight_remote INFO     c086f993-3e4a-4d50-8375-319a1912ad4f - RECEIVED: Your job has been received and is waiting approval.
2024-08-30 20:17:59,109 MainProcess nnsight_remote INFO     c086f993-3e4a-4d50-8375-319a1912ad4f - APPROVED: Your job was approved and is waiting to be run.
2024-08-30 20:17:59,135 MainProcess nnsight_remote INFO     c086f993-3e4a-4d50-8375-319a1912ad4f - RUNNING: Your job has started running.
2024-08-30 20:17:59,512 MainProcess nnsight_remote INFO     c086f993-3e4a-4d50-8375-319a1912ad4f - COMPLETED: Your job has been completed.


Use a `Session` to define the `Iterator` context and pass in a sequence of items that you want to loop over at each executed iteration.

The sequence must be iterable or be a Proxy with an iterable value.

The iterable's item can be referenced in the inner intervention body of the `Iterator`.

### loop

The `Iterator` context extends all the `nnsight` graph-based functionalities, but also closely mimics the conventional `for` loop statement in Python, which allows it to support all kind of iterative operations.

In [40]:
import nnsight

with llm.session(remote=True) as session:
  l = nnsight.list()
  [l.append(num) for num in range(0, 3)] # adding 0, 1, 2 to l
  with session.iter(l) as item: # with session.iter([0, 1, 2]) also works!
    nnsight.log(item)

2024-08-30 20:49:46,956 MainProcess nnsight_remote INFO     5c4f92fb-8961-4c41-becf-7d56c1bb58ba - RECEIVED: Your job has been received and is waiting approval.
2024-08-30 20:49:46,993 MainProcess nnsight_remote INFO     5c4f92fb-8961-4c41-becf-7d56c1bb58ba - APPROVED: Your job was approved and is waiting to be run.
2024-08-30 20:49:47,021 MainProcess nnsight_remote INFO     5c4f92fb-8961-4c41-becf-7d56c1bb58ba - RUNNING: Your job has started running.
2024-08-30 20:49:47,042 MainProcess nnsight_remote INFO     5c4f92fb-8961-4c41-becf-7d56c1bb58ba - LOG: 0
2024-08-30 20:49:47,060 MainProcess nnsight_remote INFO     5c4f92fb-8961-4c41-becf-7d56c1bb58ba - LOG: 1
2024-08-30 20:49:47,081 MainProcess nnsight_remote INFO     5c4f92fb-8961-4c41-becf-7d56c1bb58ba - LOG: 2
2024-08-30 20:49:47,126 MainProcess nnsight_remote INFO     5c4f92fb-8961-4c41-becf-7d56c1bb58ba - COMPLETED: Your job has been completed.


You can create nested `Iterator` contexts:

In [9]:
import nnsight

with llm.session() as session:
  l = nnsight.list([[10]] * 5)

  l2 = nnsight.list().save()
  with session.iter(l) as item:
    with session.iter(item) as item_2:
      l2.append(item_2)

print("List: ", l2)

List:  [10, 10, 10, 10, 10]


You can skip some iterations:

In [11]:
import nnsight

with llm.session(remote=True) as session:

  with session.iter([0, 1, 2, 3], return_context=True) as (item, iterator):
    with iterator.cond(item % 2 == 0):
      nnsight.log(item)

2024-08-30 21:03:19,288 MainProcess nnsight_remote INFO     9587a57a-6375-405a-9439-fda3e86b5f4f - RECEIVED: Your job has been received and is waiting approval.
2024-08-30 21:03:19,315 MainProcess nnsight_remote INFO     9587a57a-6375-405a-9439-fda3e86b5f4f - APPROVED: Your job was approved and is waiting to be run.
2024-08-30 21:03:19,341 MainProcess nnsight_remote INFO     9587a57a-6375-405a-9439-fda3e86b5f4f - RUNNING: Your job has started running.
2024-08-30 21:03:19,360 MainProcess nnsight_remote INFO     9587a57a-6375-405a-9439-fda3e86b5f4f - LOG: 0
2024-08-30 21:03:19,379 MainProcess nnsight_remote INFO     9587a57a-6375-405a-9439-fda3e86b5f4f - LOG: 2
2024-08-30 21:03:19,425 MainProcess nnsight_remote INFO     9587a57a-6375-405a-9439-fda3e86b5f4f - COMPLETED: Your job has been completed.


Or, you can choose to `break` out of the Iteration loop early:

In [12]:
import nnsight

with llm.session(remote=True) as session:

  with session.iter([0, 1, 2, 3], return_context=True) as (item, iterator):
      with iterator.cond(item == 2):
        iterator.exit()

      nnsight.log(item)

2024-08-30 21:03:37,096 MainProcess nnsight_remote INFO     4269ef5e-0e34-4070-8f9d-cec8460f5069 - RECEIVED: Your job has been received and is waiting approval.
2024-08-30 21:03:37,126 MainProcess nnsight_remote INFO     4269ef5e-0e34-4070-8f9d-cec8460f5069 - APPROVED: Your job was approved and is waiting to be run.
2024-08-30 21:03:37,152 MainProcess nnsight_remote INFO     4269ef5e-0e34-4070-8f9d-cec8460f5069 - RUNNING: Your job has started running.
2024-08-30 21:03:37,175 MainProcess nnsight_remote INFO     4269ef5e-0e34-4070-8f9d-cec8460f5069 - LOG: 0
2024-08-30 21:03:37,194 MainProcess nnsight_remote INFO     4269ef5e-0e34-4070-8f9d-cec8460f5069 - LOG: 1
2024-08-30 21:03:37,234 MainProcess nnsight_remote INFO     4269ef5e-0e34-4070-8f9d-cec8460f5069 - COMPLETED: Your job has been completed.


The `Iterator` context is a niece piece of functionality that allows you to define a bunch of basic code operations that can now be "traceable" by `nnsight`.

But in what kind of experimental scenario would someone even need to use it?

In the next section, we delve into a powerful use case of the `Iterator` context and see how it enables it!